<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/3_2%EB%B2%88_In_Selection_%EC%A0%95%EA%B7%9C%EC%8B%9D%ED%99%9C%EC%9A%A9_Generating_Features_Time_to12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')



CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


## Train 으로부터 mytrain 데이터 생성

In [0]:
Selection = train.loc[train['event']=='Selection',:]
Selection['event_contents'].value_counts()

[]                                                                                                                                                                                                                                                                                  811772
['OrbitalCommand [3100001]']                                                                                                                                                                                                                                                         75165
['Nexus [3100001]']                                                                                                                                                                                                                                                                  55956
['OrbitalCommand [3440001]']                                                                                                                           

In [0]:
import re
def SUB(x):

    if str(x) != "[]":
        result = re.sub("[',]","",x)
        result = re.sub(' ','', result)
        result = result[1:-1]
        result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
        #result = result.split(',')
    else:
        result = "Empty"

    result = result + ','
    return result

Selection['event_contents'] = Selection.apply(lambda x: SUB(x['event_contents']), axis=1)
Selection['event_contents'].value_counts()

Egg[],                                                                                                                                                                                                                                                                                                                                                                                                                                           1234596
Empty,                                                                                                                                                                                                                                                                                                                                                                                                                                            811772
SCV[],                                                                                                                

In [0]:
game_id = train.game_id.unique().tolist()
columns = ['text0','text1','winner']
mytrain = np.zeros(len(game_id)*len(columns))
mytrain = mytrain.reshape(len(game_id),len(columns))
mytrain = pd.DataFrame(mytrain, columns=columns)
mytrain.index = game_id
mytrain.winner = train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
mytrain.loc[:,'text0'] = Selection.loc[Selection['player']==0,:].loc[Selection['event']=='Selection',:].groupby(['game_id']).event_contents.sum()
mytrain.loc[:,'text1'] = Selection.loc[Selection['player']==1,:].loc[Selection['event']=='Selection',:].groupby(['game_id']).event_contents.sum()
del train
del Selection
mytrain = mytrain.fillna('')


## Test 로부터 mytest 데이터 생성

In [0]:
test = pd.read_csv('test.csv')
Selection = test.loc[test['event']=='Selection',:]

Selection['event_contents'] = Selection.apply(lambda x: SUB(x['event_contents']), axis=1)
Selection['event_contents'].value_counts()

Egg[],                                                                                                                                                                                                                                                                                                                                                                                                           527093
Empty,                                                                                                                                                                                                                                                                                                                                                                                                           345691
SCV[],                                                                                                                                                                                  

In [0]:
game_id = test.game_id.unique().tolist()
columns = ['text0','text1']
mytest = np.zeros(len(game_id)*len(columns))
mytest = mytest.reshape(len(game_id),len(columns))
mytest = pd.DataFrame(mytest, columns=columns)
mytest.index = game_id

mytest.loc[:,'text0'] = Selection.loc[Selection['player']==0,:].loc[Selection['event']=='Selection',:].groupby(['game_id']).event_contents.sum()
mytest.loc[:,'text1'] = Selection.loc[Selection['player']==1,:].loc[Selection['event']=='Selection',:].groupby(['game_id']).event_contents.sum()
del test
del Selection
mytest = mytest.fillna('')


### 1. CountVevtorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 2
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
#del player0, player1


(38872, 171)
(38872, 171)
(38872, 342)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
min_df = 2
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

#del player0, player1

(16787, 165)
(16787, 166)
(16787, 331)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

342
331
329


In [0]:

###################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=seed).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포 #  min_df=1 : 0.6792 vs min_df=2 : 0.6790
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM #  min_df=1 : 0.6757 vs min_df=2 : 0.6757
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.6790
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000,

In [0]:
X.to_pickle('train_Selection12.pkl')
X_test.to_pickle('test_Selection12.pkl')